Computing Project - Monte carlo Simulation of fluids

In [36]:
import numpy as np
import math
import matplotlib.pyplot as plt
import json
import re

In [37]:
%matplotlib qt

In [38]:
#load config parameters
def load_jsonc(path):
    with open(path, "r", encoding="utf-8") as f:
        text = f.read()

    # remove // comments
    text = re.sub(r"//.*", "", text)
    # remove /* block comments */
    text = re.sub(r"/\\*.*?\\*/", "", text, flags=re.DOTALL)

    return json.loads(text)

config = load_jsonc("config.jsonc")
N   = config["N"]
L   = config["L"]
eta = config["eta"]


print(config)


{'N': 100, 'eta': 0.68, 'L': 100, 'dim': 2, 'equil_steps': 50000, 'prod_steps': 500000, 'sample_every': 1000, 'tune_every': 100, 'r_max': 5.0, 'gr_bins': 200, 'lattice': 'square', 'seed': 12345}


In [39]:
#Computing box size from N, density, and particle diameter. Maximum particle diameter of 1? Why not define box as sqrt(N)?
def sigma_from_eta_2d(N: int, L: float, eta: float) -> float:
    """
    From 2D packing fraction:
      eta = N * (pi * sigma^2 / 4) / L^2
      -> sigma = 2 L sqrt(eta / (N pi))
    """
    return 2.0 * L * math.sqrt(eta / (N * math.pi))

In [40]:
def balanced_grid_shape_2d(N: int) -> tuple[int, int]:
    """
    Choose an integer grid (nx, ny) with nx*ny >= N and as close to square as possible.
    """
    n = math.ceil(math.sqrt(N))
    return n, n  # simple and near-optimal for most N

In [41]:
def square_lattice_positions_2d(N: int, L: float) -> tuple[np.ndarray, tuple[float, float]]:
    """
    Return (N, 2) positions on a square grid, cell-centered so all disks lie fully inside [0, L]^2.
    Also return the spacings (ax, ay) between neighboring centers.
    """
    nx, ny = balanced_grid_shape_2d(N)
    ax, ay = L / nx, L / ny
    coords = []
    for j in range(ny):
        y = (j + 0.5) * ay
        for i in range(nx):
            x = (i + 0.5) * ax
            coords.append((x, y))
    return np.array(coords[:N], dtype=float), (ax, ay)

In [42]:
def initialize_cubic_lattice_2d(N: int, L: float, eta: float):
    """
    Initialize N particles on a 2D cubic (square) lattice in a fixed box of side L.
    Particle diameter sigma is determined by the packing fraction eta.
    Ensures no overlaps (spacing >= sigma). Returns (positions, sigma, eta_actual, (ax, ay)).
    """
    # 1) derive particle size from (N, L, eta)
    sigma = sigma_from_eta_2d(N, L, eta)

    # 2) place on a cell-centered square grid
    positions, (ax, ay) = square_lattice_positions_2d(N, L)

    # 3) feasibility: minimal center spacing must be >= sigma
    a_min = min(ax, ay)
    if a_min < sigma - 1e-12:
        # Maximum feasible eta with this grid (touching, sigma=a_min):
        eta_max_touching = (N * math.pi * (a_min**2) / 4.0) / (L**2)
        raise ValueError(
            f"Infeasible: target η={eta:.6g} ⇒ σ={sigma:.6g}, but grid spacing a_min={a_min:.6g} < σ (overlap).\n"
            f"Lower η to ≤ ~{eta_max_touching:.6g}, increase L, or reduce N."
        )

    # 4) actual eta (equals requested eta by construction)
    eta_actual = (N * math.pi * sigma**2 / 4.0) / (L**2)
    return positions, sigma, eta_actual, (ax, ay)

In [43]:
positions, sigma, eta_actual, spacings = initialize_cubic_lattice_2d(N, L, eta)

In [44]:
print("Loaded from config:")
print(f"N   = {N}")
print(f"L   = {L}")
print(f"eta = {eta}")

print("\nComputed:")
print(f"sigma = {sigma}")
print(f"actual packing = {eta_actual}")
print(f"grid spacing = {spacings}")

print("\nFirst 5 particles:\n", positions[:5])

Loaded from config:
N   = 100
L   = 100
eta = 0.68

Computed:
sigma = 9.304852983362556
actual packing = 0.68
grid spacing = (10.0, 10.0)

First 5 particles:
 [[ 5.  5.]
 [15.  5.]
 [25.  5.]
 [35.  5.]
 [45.  5.]]


In [45]:
def plot_disks_LxL(positions: np.ndarray, L: float, sigma: float, title: str = ""):
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.set_aspect("equal")
    ax.set_xlim(0, L)
    ax.set_ylim(0, L)
    ax.set_xticks([]); ax.set_yticks([])
    ax.set_position([0, 0, 1, 1])  # remove all margins
    if title:
        ax.set_title(title, pad=8)

    # draw disks fully inside box
    r = sigma / 2.0
    for (x, y) in positions:
        ax.add_patch(plt.Circle((x, y), r, fc="navy", ec="black", lw=0.6))

    # box outline
    ax.plot([0, L, L, 0, 0], [0, 0, L, L, 0], "k-", lw=1)
    plt.show()

In [46]:
plot_disks_LxL(
        positions,
        L,
        sigma,
        title=f"Cubic (square) lattice — N={N}, η={eta}, L={L}"
    )

In [47]:
# --- PBC Visualization Function ---
def plot_disks_LxL_pbc(positions: np.ndarray, L: float, sigma: float, title: str = ""):
    """
    Plot disks in the box [0,L]^2 plus their periodic 'ghosts'
    to visualize continuity across edges.
    """
    import matplotlib.pyplot as plt

    r = 0.5 * sigma
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.set_aspect("equal")
    ax.set_xlim(0, L)
    ax.set_ylim(0, L)
    ax.set_xticks([])
    ax.set_yticks([])
    if title:
        ax.set_title(title, pad=8)

    # Draw simulation box outline
    ax.plot([0, L, L, 0, 0], [0, 0, L, L, 0], "k-", lw=1)

    # Draw each disk and its periodic copies
    offsets = (-L, 0.0, L)
    for (x, y) in positions:
        for dx in offsets:
            X = x + dx
            if X < -r or X > L + r:
                continue
            for dy in offsets:
                Y = y + dy
                if Y < -r or Y > L + r:
                    continue
                circ = plt.Circle((X, Y), r, fc="navy", ec="black", lw=0.6, clip_on=True)
                ax.add_patch(circ)

    plt.show()


In [48]:
#wrap boundary conditions
def wrap_pbc(r: np.ndarray, L: float) -> np.ndarray:
    """Wrap coordinates into [0, L)."""
    r_wrapped = r.copy()
    r_wrapped -= L * np.floor(r_wrapped / L)
    return r_wrapped

def min_image(dr: np.ndarray, L: float) -> np.ndarray:
    """Apply minimum image convention to a displacement vector."""
    return dr - L * np.round(dr / L)


In [49]:
import seaborn as sns

def draw_disks(ax, positions, L, sigma, pbc: bool):
    r = 0.5 * sigma
    ax.set_aspect("equal")
    ax.set_xlim(0, L)
    ax.set_ylim(0, L)
    ax.set_xticks([]); ax.set_yticks([])
    ax.plot([0, L, L, 0, 0], [0, 0, L, L, 0], "k-", lw=1)

    # ---- COLUMN COLORING WITH SEABORN ----
    xs = positions[:, 0]
    x_norm = (xs - xs.min()) / (xs.max() - xs.min() + 1e-12)

    # choose a seaborn palette:
    # examples: "viridis", "magma", "crest", "rocket", "icefire", "flare"
    palette = sns.color_palette("mako", as_cmap=True)
    # palette = sns.color_palette("rocket", as_cmap=True)
    # palette = sns.color_palette("viridis", as_cmap=True)

    # no PBC ghosts
    if not pbc:
        for (x, y), xn in zip(positions, x_norm):
            color = palette(xn)
            ax.add_patch(plt.Circle((x, y), r, fc=color, ec="black", lw=0.3))
        return

    # PBC ghost drawing
    offsets = (-L, 0.0, L)
    for (x, y), xn in zip(positions, x_norm):
        color = palette(xn)
        for dx in offsets:
            X = x + dx
            if X < -r or X > L + r: continue
            for dy in offsets:
                Y = y + dy
                if Y < -r or Y > L + r: continue
                ax.add_patch(plt.Circle((X, Y), r, fc=color, ec="black", lw=0.3))


In [50]:
def has_overlap_vec(i: int, trial_pos: np.ndarray,
                    positions: np.ndarray, sigma: float, L: float) -> bool:
    sig2 = sigma * sigma

    # displacement to all particles
    dr = trial_pos - positions              # shape (N, 2)

    # minimum image on the whole array
    dr -= L * np.round(dr / L)

    # squared distances
    d2 = dr[:, 0]*dr[:, 0] + dr[:, 1]*dr[:, 1]

    # ignore self
    d2[i] = np.inf

    # any overlap?
    return np.any(d2 < sig2)


In [51]:
def mc_step(positions: np.ndarray, sigma: float, L: float, d: float, rng: np.random.Generator) -> bool:
    N = len(positions)
    i = rng.integers(N)
    disp = d * (rng.random(2) - 0.5)
    trial = wrap_pbc(positions[i] + disp, L)
    if not has_overlap_vec(i, trial, positions, sigma, L):
        positions[i] = trial     # accept
        return True
    return False 

In [52]:
def tune_stepsize(d: float, accept_rate: float, up: float = 1.05, down: float = 0.95,
                  d_min: float = 1e-4, d_max: float = np.inf) -> float:
    if accept_rate > 0.50:
        d *= up
    elif accept_rate < 0.25:
        d *= down
    return float(np.clip(d, d_min, d_max))

In [53]:
# --- Visualization during Monte Carlo ---

def plot_snapshot(positions, L, sigma, step, ax=None):
    """Plot disks for a given MC step (reusing axes for speed)."""
    if ax is None:
        fig, ax = plt.subplots(figsize=(6, 6))
        ax.set_aspect("equal")
        ax.set_xlim(0, L)
        ax.set_ylim(0, L)
        ax.set_xticks([]); ax.set_yticks([])
    ax.clear()
    ax.set_xlim(0, L)
    ax.set_ylim(0, L)
    ax.set_title(f"Monte Carlo step {step}")
    
    r = sigma / 2.0
    for (x, y) in positions:
        ax.add_patch(plt.Circle((x, y), r, fc="navy", ec="black", lw=0.6))
    ax.plot([0, L, L, 0, 0], [0, 0, L, L, 0], "k-", lw=1)
    plt.pause(0.001)
    return ax


In [54]:
def run_mc(positions: np.ndarray,
           sigma: float,
           L: float,
           d_init: float,
           equil_steps: int,
           prod_steps: int,
           tune_every: int,
           rng: np.random.Generator,
           plot_every: int | None = None,      # e.g., 100 for live snapshots; None to disable
           save_frames: bool = False,          # save PNGs during the run
           frames_dir: str = "frames_mc",      # where PNGs go
           cap_sigma_frac: float = 0.5,        # cap step size: d <= cap_sigma_frac * sigma
           stitch_gif: bool = False,           # stitch PNGs into a GIF at the end
           gif_out: str = "mc_evolution.gif",
           fps: int = 15):
    """
    Single-particle Metropolis MC with step-size auto-tuning.

    Returns:
      positions        : final positions (in-place updated)
      d_final          : final step size
      acc_rates        : np.array of acceptance rates measured every `tune_every` steps
      d_history        : np.array of d after each tuning
      totals           : dict with accepted, rejected, attempted, acceptance_rate
    """
    import os
    import matplotlib.pyplot as plt

    # ---- helper: capped tuner to avoid runaway step sizes
    def tune_stepsize_capped(d: float, accept_rate: float,
                         target: float = 0.4, k: float = 1.2,
                         d_min: float = 1e-4) -> float:
        # multiplicative update toward target
        d *= float(np.exp(k * (accept_rate - target)))

        # cap based on mean spacing, not sigma
        mean_spacing = L / np.sqrt(len(positions))  # ≈ interparticle distance
        d_max = 0.8 * mean_spacing                  # tune 0.5–1.0 as you prefer

        return float(np.clip(d, d_min, d_max))


    # ---- prep
    if save_frames:
        os.makedirs(frames_dir, exist_ok=True)

    total_steps = int(equil_steps + prod_steps)
    d = float(d_init)

    # counters
    window_accepted = 0
    window_attempted = 0
    total_accepted = 0
    total_attempted = 0

    acc_rates: list[float] = []
    d_history: list[float] = []

    # live plot (reuse axes for speed)
    live_fig = None
    live_ax = None

    # ---- main loop
    for step in range(1, total_steps + 1):
        # one MC move
        accepted = mc_step(positions, sigma, L, d, rng)  # bool
        window_accepted += int(accepted)
        window_attempted += 1
        total_accepted  += int(accepted)
        total_attempted += 1

        # tune step size
        if step % tune_every == 0:
            acc_rate = (window_accepted / window_attempted) if window_attempted else 0.0
            d = tune_stepsize_capped(d, acc_rate)
            acc_rates.append(acc_rate)
            d_history.append(d)
            window_accepted = 0
            window_attempted = 0

        # live snapshot + optional frame save
        if plot_every and (step % plot_every == 0):
            # live display (ghosted view)
            if live_ax is None:
                live_fig, live_ax = plt.subplots(figsize=(6, 6))
            live_ax.clear()
            live_ax.set_title(f"Monte Carlo step {step}")
            draw_disks(live_ax, positions, L, sigma, pbc=True)
            plt.pause(0.001)

            # save PNG frame (ghosted view)
            if save_frames:
                fig2, ax2 = plt.subplots(figsize=(5, 5))
                ax2.set_title(f"Step {step}")
                draw_disks(ax2, positions, L, sigma, pbc=True)
                out_path = os.path.join(frames_dir, f"frame_{step:07d}.png")
                fig2.savefig(out_path, dpi=150, bbox_inches="tight")
                plt.close(fig2)

    if live_fig is not None:
        plt.show()

    totals = {
        "accepted": int(total_accepted),
        "attempted": int(total_attempted),
        "rejected": int(total_attempted - total_accepted),
        "acceptance_rate": float(total_accepted / total_attempted) if total_attempted else 0.0,
    }

    # optional: stitch frames into a GIF at the end
    if stitch_gif and save_frames:
        try:
            import imageio.v2 as imageio
            files = sorted(
                os.path.join(frames_dir, f) for f in os.listdir(frames_dir)
                if f.lower().endswith(".png")
            )
            if files:
                imgs = [imageio.imread(f) for f in files]
                imageio.mimsave(gif_out, imgs, duration=1.0 / fps, loop=0)
                print(f"[run_mc] Saved GIF: {gif_out}")
            else:
                print(f"[run_mc] No PNG frames in {frames_dir}; GIF skipped.")
        except Exception as e:
            print(f"[run_mc] GIF stitching skipped ({e}). Install `imageio` if needed.")

    return positions, d, np.array(acc_rates), np.array(d_history), totals


In [55]:
import os

def stitch_frames(frames_dir: str,
                  gif_out: str = "mc_evolution2.gif",
                  mp4_out: str | None = "mc_evolution2.mp4",
                  fps: int = 15):
    import imageio.v2 as imageio
    files = sorted(
        os.path.join(frames_dir, f) for f in os.listdir(frames_dir)
        if f.lower().endswith(".png")
    )
    if not files:
        print(f"No PNG frames in {frames_dir}.")
        return
    imgs = [imageio.imread(f) for f in files]
    imageio.mimsave(gif_out, imgs, duration=1.0/fps, loop=0)
    print("Saved GIF:", gif_out)
    if mp4_out:
        try:
            import imageio_ffmpeg  # noqa
            w = imageio.get_writer(mp4_out, fps=fps, codec="libx264", quality=8)
            for im in imgs: w.append_data(im)
            w.close()
            print("Saved MP4:", mp4_out)
        except Exception as e:
            print("MP4 skipped:", e)


In [419]:
rng = np.random.default_rng(config["seed"])
d0 = 0.2 * sigma

positions_run, d_final, acc_hist, d_hist, totals = run_mc(
    positions=positions.copy(),
    sigma=sigma,
    L=L,
    d_init=d0,
    equil_steps=10000,
    prod_steps=0,
    tune_every=config["tune_every"],
    rng=rng,
    plot_every=100,
    save_frames=True,
    frames_dir="frames_mc",
    cap_sigma_frac=0.5,
    stitch_gif=True,                 # auto-make GIF at the end (requires imageio)
    gif_out="mc_evolution.gif",
    fps=15
)

print("Final tuned d:", d_final)
print("Moves — accepted:", totals["accepted"],
      "rejected:", totals["rejected"],
      "attempted:", totals["attempted"],
      f"acceptance: {100*totals['acceptance_rate']:.1f}%")


[run_mc] Saved GIF: mc_evolution.gif
Final tuned d: 20.732100974535104
Moves — accepted: 4009 rejected: 5991 attempted: 10000 acceptance: 40.1%


In [56]:
def run_mc_first_last(positions: np.ndarray,
                      sigma: float,
                      L: float,
                      d_init: float,
                      n_steps: int,
                      rng: np.random.Generator,
                      cap_sigma_frac: float | None = 0.5):
    """
    Runs n_steps of MC and plots ONLY the first and last configurations.
    Returns final positions and final step size d.
    """

    import matplotlib.pyplot as plt

    # --- Cap step size relative to sigma ---
    if cap_sigma_frac is not None:
        d = min(float(d_init), cap_sigma_frac * sigma)
    else:
        d = float(d_init)

    # --- Plot FIRST frame ---
    fig1, ax1 = plt.subplots(figsize=(6, 6))
    ax1.set_title("Initial configuration")
    draw_disks(ax1, positions, L, sigma, pbc=True)
    plt.show()

    # --- MC loop (no plotting inside) ---
    N = len(positions)
    total_acc = 0

    for step in range(1, n_steps + 1):
        accepted = mc_step(positions, sigma, L, d, rng)
        total_acc += int(accepted)

    # --- Plot LAST frame ---
    fig2, ax2 = plt.subplots(figsize=(6, 6))
    ax2.set_title(f"Final configuration after {n_steps} steps")
    draw_disks(ax2, positions, L, sigma, pbc=True)
    plt.show()

    acc_rate = total_acc / n_steps
    print(f"Final acceptance: {100*acc_rate:.2f}%")
    print(f"Final d: {d}")

    return positions, d


In [57]:
rng = np.random.default_rng(config["seed"])
d0 = 0.2 * sigma   # or whatever you prefer

positions_final, d_final = run_mc_first_last(
    positions=positions.copy(),
    sigma=sigma,
    L=L,
    d_init=d0,
    n_steps=10000,        # total MC steps
    rng=rng,
    cap_sigma_frac=0.5
)


Final acceptance: 37.25%
Final d: 1.8609705966725114


In [58]:
def accumulate_gr_2d(positions: np.ndarray,
                     L: float,
                     sigma: float,
                     r_max_sigma: float,
                     nbins: int,
                     gr_counts: np.ndarray) -> None:
    """
    Accumulate pair-distance histogram counts into gr_counts for one snapshot.

    positions : (N, 2) array
    L         : box length
    sigma     : particle diameter
    r_max_sigma : max r in units of sigma (e.g., 5.0)
    nbins     : number of radial bins
    gr_counts : array of length nbins, modified in-place
    """
    N = len(positions)
    r_max = r_max_sigma * sigma
    edges = np.linspace(0.0, r_max, nbins + 1)

    # loop over reference particles
    for i in range(N):
        ri = positions[i]
        # vectorized min-image distances to all particles
        dr_vecs = positions - ri                # (N, 2)
        dr_vecs -= L * np.round(dr_vecs / L)    # minimum image on whole array
        d2 = np.sum(dr_vecs**2, axis=1)
        d2[i] = np.inf                          # ignore self
        r = np.sqrt(d2)

        mask = r < r_max
        if np.any(mask):
            hist, _ = np.histogram(r[mask], bins=edges)
            gr_counts += hist


In [59]:
def finalize_gr_2d(gr_counts: np.ndarray,
                   N: int,
                   L: float,
                   sigma: float,
                   r_max_sigma: float,
                   nbins: int,
                   n_frames: int):
    """
    Convert accumulated gr_counts into normalized g(r) and r array.
    """
    r_max = r_max_sigma * sigma
    edges = np.linspace(0.0, r_max, nbins + 1)
    r_centers = 0.5 * (edges[:-1] + edges[1:])
    dr = edges[1] - edges[0]
    rho = N / (L * L)

    # expected counts for ideal gas: N * rho * 2π r dr per frame
    norm = n_frames * N * rho * (2.0 * np.pi) * r_centers * dr

    with np.errstate(divide="ignore", invalid="ignore"):
        g_r = np.where(norm > 0, gr_counts / norm, 0.0)

    return r_centers, g_r


In [60]:
def run_full_mc_with_gr(config, positions_init, sigma, L):
    """
    Equilibrate, then run production MC and sample g(r).
    Assumes 2D hard disks, N fixed, L fixed.
    Uses:
      - config["equil_steps"]
      - config["prod_steps"]
      - config["tune_every"]
      - config["sample_every"]
      - config["r_max"]  (in units of sigma, e.g. 5.0)
      - config["gr_bins"]
      - config["seed"]
    """
    N = config["N"]
    equil_steps = config["equil_steps"]
    prod_steps = config["prod_steps"]
    tune_every = config["tune_every"]
    sample_every = config["sample_every"]
    r_max_sigma = config["r_max"]
    nbins = config["gr_bins"]
    seed = config["seed"]

    rng = np.random.default_rng(seed)

    # --- 1) Equilibration (no g(r) sampling) ---
    d0 = 0.2 * sigma  # or something based on spacing
    positions_eq, d_eq, acc_hist_eq, d_hist_eq, totals_eq = run_mc(
        positions=positions_init.copy(),
        sigma=sigma,
        L=L,
        d_init=d0,
        equil_steps=equil_steps,
        prod_steps=0,
        tune_every=tune_every,
        rng=rng,
        plot_every=None,     # no plotting for speed
        save_frames=False,
        cap_sigma_frac=0.5,  # or None if you want pure 5% tuning
        stitch_gif=False
    )

    print("Equilibration done.")
    print(f"  Equil acceptance: {100*totals_eq['acceptance_rate']:.1f}%")

    # --- 2) Production + g(r) sampling ---
    gr_counts = np.zeros(nbins, dtype=float)
    n_frames = 0

    d = float(d_eq)
    window_acc = 0
    window_att = 0

    # local tuner (same 5% rule)
    def tune_stepsize_capped(d: float, accept_rate: float,
                             up: float = 1.05, down: float = 0.95,
                             d_min: float = 1e-4) -> float:
        if accept_rate > 0.50:
            d *= up
        elif accept_rate < 0.25:
            d *= down
        d_max = 0.5 * sigma
        d = min(d, d_max)
        return max(d, d_min)

    for step in range(1, prod_steps + 1):
        accepted = mc_step(positions_eq, sigma, L, d, rng)
        window_acc += int(accepted)
        window_att += 1

        # tune during production as well
        if step % tune_every == 0:
            acc_rate = window_acc / window_att if window_att else 0.0
            d = tune_stepsize_capped(d, acc_rate)
            window_acc = 0
            window_att = 0

        # sample g(r)
        if step % sample_every == 0:
            accumulate_gr_2d(
                positions=positions_eq,
                L=L,
                sigma=sigma,
                r_max_sigma=r_max_sigma,
                nbins=nbins,
                gr_counts=gr_counts
            )
            n_frames += 1

    print("Production done.")
    print(f"  g(r) sampled {n_frames} times.")

    # --- 3) Normalize g(r) ---
        # --- 3) Normalize g(r) ---
    r, g_r = finalize_gr_2d(
        gr_counts=gr_counts,
        N=N,
        L=L,
        sigma=sigma,
        r_max_sigma=r_max_sigma,
        nbins=nbins,
        n_frames=n_frames
    )

    # --- 4) Pretty plot ---
    plot_gr_clean(r, g_r, config["eta"])

    return positions_eq, r, g_r


import seaborn as sns

def plot_gr_clean(r, g_r, eta):
    """
    Clean, publication-style plot of g(r):
      - Origin (0,0) visible
      - No background
      - Black line
      - Seaborn styling
      - No top/right spines
    """

    # Seaborn style (no grid, clean look)
    sns.set_style("white")  
    sns.set_context("talk")

    fig, ax = plt.subplots(figsize=(7, 5))

    # Remove background patch
    fig.patch.set_alpha(0.0)
    ax.set_facecolor("white")

    # Plot g(r)
    ax.plot(r / 1.0, g_r, color="black", linewidth=2)

    # Axes labels
    ax.set_xlabel(r"$r/\sigma$")
    ax.set_ylabel(r"$g(r)$")
    ax.set_title(rf"Pair correlation function $g(r)$ at $\eta = {eta}$")

    # Force bottom-left corner to show (0,0)
    ax.set_xlim(0, r.max())
    ax.set_ylim(0, max(g_r)*1.05)

    # Remove top/right spines
    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)

    # Make sure left/bottom spines are clean
    ax.spines["left"].set_linewidth(1.1)
    ax.spines["bottom"].set_linewidth(1.1)

    plt.tight_layout()
    plt.show()



In [431]:
# assuming you've already done:
# config = load_jsonc("config.jsonc")
# positions, sigma, eta_actual, spacings = initialize_cubic_lattice_2d(N, L, eta)

positions_final, r, g_r = run_full_mc_with_gr(config, positions, sigma, L)

# plot g(r) vs r/σ
plt.figure(figsize=(6,4))
plt.plot(r / sigma, g_r, '-')
plt.xlabel(r"$r/\sigma$")
plt.ylabel(r"$g(r)$")
plt.xlim(0.0, 5.0)
plt.grid(True, alpha=0.2)
plt.title("Pair correlation function $g(r)$ at $\eta = 0.68$")
plt.show()


Equilibration done.
  Equil acceptance: 40.0%
Production done.
  g(r) sampled 500 times.


In [61]:
def sample_gr_2d(positions: np.ndarray,
                 L: float,
                 sigma: float,
                 r_max_sigma: float,
                 nbins: int) -> tuple[np.ndarray, np.ndarray]:
    """
    Compute a *single-frame* estimate of g(r) histogram (unnormalized counts).

    Returns:
      r_centers : (nbins,) bin centers (absolute units)
      counts    : (nbins,) counts in each shell for this snapshot
    """
    N = len(positions)
    r_max = r_max_sigma * sigma
    edges = np.linspace(0.0, r_max, nbins + 1)
    r_centers = 0.5 * (edges[:-1] + edges[1:])

    counts = np.zeros(nbins, dtype=float)

    # loop over i < j to avoid double counting
    for i in range(N - 1):
        dr = positions[i+1:] - positions[i]          # shape (N-i-1, 2)
        dr -= L * np.round(dr / L)                   # minimum image
        d2 = np.einsum("ij,ij->i", dr, dr)           # squared distances
        r = np.sqrt(d2)
        mask = r < r_max
        if np.any(mask):
            hist, _ = np.histogram(r[mask], bins=edges)
            counts += hist

    return r_centers, counts


In [62]:
def run_with_gr_evolution(positions: np.ndarray,
                          sigma: float,
                          L: float,
                          d_init: float,
                          equil_steps: int,
                          prod_steps: int,
                          sample_every: int,
                          r_max_sigma: float,
                          nbins: int,
                          rng: np.random.Generator,
                          tune_every: int,
                          cap_sigma_frac: float | None = 0.5):
    """
    Full MC run (equil + production) that accumulates g(r) and stores the
    cumulative g(r) curve after each sample for animation.

    Returns:
      r            : (nbins,) bin centers in units of sigma
      g_list       : list of g(r) arrays over time (len = #samples)
      positions    : final particle positions
    """
    N = len(positions)
    rho = N / (L * L)

    # --- step size with optional cap ---
    if cap_sigma_frac is not None:
        d = min(float(d_init), cap_sigma_frac * sigma)
    else:
        d = float(d_init)

    # --- equilibration (no sampling) ---
    for step in range(1, equil_steps + 1):
        mc_step(positions, sigma, L, d, rng)
        if (step % tune_every) == 0:
            # simple acceptance-based tuning could be added here if you like;
            # to keep this compact, we keep d fixed, which is ok if you've
            # tuned it beforehand.
            pass

    # --- production + g(r) accumulation ---
    r_abs, counts_global = sample_gr_2d(
        positions, L, sigma, r_max_sigma, nbins
    )  # first frame just to get r_abs shape
    counts_global[:] = 0.0
    n_samples = 0
    g_list: list[np.ndarray] = []

    edges = np.linspace(0.0, r_max_sigma * sigma, nbins + 1)
    dr = edges[1] - edges[0]

    for step in range(1, prod_steps + 1):
        mc_step(positions, sigma, L, d, rng)

        # optionally tune d here every tune_every as in your run_mc

        if (step % sample_every) == 0:
            _, counts = sample_gr_2d(
                positions, L, sigma, r_max_sigma, nbins
            )
            counts_global += counts
            n_samples += 1

            # cumulative g(r) so far
            shell_area = 2.0 * math.pi * r_abs * dr  # 2D
            ideal_counts_per_sample = rho * shell_area * N  # i<j counted once
            norm = n_samples * ideal_counts_per_sample
            with np.errstate(divide="ignore", invalid="ignore"):
                g = np.where(norm > 0, counts_global / norm, 0.0)

            g_list.append(g.copy())

    # convert r to units of sigma for plotting
    r_over_sigma = r_abs / sigma
    return r_over_sigma, g_list, positions



In [63]:
def animate_gr_evolution(r_over_sigma: np.ndarray,
                         g_list: list[np.ndarray],
                         eta: float,
                         sample_every: int):
    """
    Simple animation of g(r) evolving over time.
    Assumes g_list[k] is the cumulative g(r) after (k+1)*sample_every steps.
    """
    import matplotlib.pyplot as plt

    fig, ax = plt.subplots(figsize=(6, 4))
    line, = ax.plot(r_over_sigma, g_list[0])
    ax.set_xlim(0, r_over_sigma[-1])
    ymax = max(g.max() for g in g_list) * 1.1
    ax.set_ylim(0, ymax)

    ax.set_xlabel(r"$r/\sigma$")
    ax.set_ylabel(r"$g(r)$")

    for idx, g in enumerate(g_list):
        line.set_ydata(g)
        step = (idx + 1) * sample_every
        ax.set_title(fr"$g(r)$ at $\eta={eta}$, after {step:.0f} MC steps")
        plt.pause(0.1)

    plt.show()


In [427]:
rng = np.random.default_rng(config["seed"])

# start from your square lattice
positions, sigma, eta_actual, spacings = initialize_cubic_lattice_2d(N, L, eta)

r, g_list, positions_final = run_with_gr_evolution(
    positions=positions,
    sigma=sigma,
    L=L,
    d_init=0.2 * sigma,
    equil_steps=config["equil_steps"],      # e.g. 10000
    prod_steps=config["prod_steps"],        # e.g. 500000
    sample_every=config["sample_every"],    # e.g. 1000
    r_max_sigma=config["r_max"],            # 5.0
    nbins=config["gr_bins"],                # 200
    rng=rng,
    tune_every=config["tune_every"],
    cap_sigma_frac=0.5,
)

animate_gr_evolution(r, g_list, eta, config["sample_every"])


In [65]:
import os
import numpy as np
import math
import matplotlib.pyplot as plt

def compute_gr_snapshot(positions: np.ndarray,
                        L: float,
                        sigma: float,
                        r_max_sigma: float,
                        nbins: int) -> tuple[np.ndarray, np.ndarray]:
    """
    Compute instantaneous g(r) for a single configuration (2D, hard disks).
    Returns (r_centers, g_r).
    """
    N = len(positions)
    rho = N / (L * L)

    r_max_abs = r_max_sigma * sigma
    edges = np.linspace(0.0, r_max_abs, nbins + 1)
    r_centers = 0.5 * (edges[:-1] + edges[1:])
    dr = edges[1] - edges[0]

    gr_counts = np.zeros(nbins, dtype=float)

    # accumulate ordered pairs over all i
    for i in range(N):
        ri = positions[i]
        dr_vecs = positions - ri                    # (N,2)
        dr_vecs -= L * np.round(dr_vecs / L)        # minimum image on whole array
        d2 = np.sum(dr_vecs**2, axis=1)
        d2[i] = np.inf                              # ignore self
        r = np.sqrt(d2)
        mask = r < r_max_abs
        if np.any(mask):
            hist, _ = np.histogram(r[mask], bins=edges)
            gr_counts += hist

    # normalisation: ideal-gas counts per bin per frame: N * rho * 2π r dr
    shell_areas = 2.0 * math.pi * r_centers * dr
    ideal_counts = N * rho * shell_areas
    with np.errstate(divide="ignore", invalid="ignore"):
        g_r = np.where(ideal_counts > 0, gr_counts / ideal_counts, 0.0)

    return r_centers, g_r


def run_full_sim_and_animate_gr(config_path: str = "config.jsonc",
                                frames_dir: str = "frames_gr",
                                gif_out: str = "gr_evolution.gif",
                                fps: int = 10):
    """
    Uses config.jsonc:
      - equilibrates
      - runs production
      - samples g(r) every sample_every
      - saves PNG frames of g(r)
      - stitches them into a GIF (gr_evolution.gif)
    """

    # --- load config ---
    cfg = load_jsonc(config_path)
    N        = int(cfg["N"])
    L        = float(cfg["L"])
    eta      = float(cfg["eta"])
    eq_steps = int(cfg["equil_steps"])
    pr_steps = int(cfg["prod_steps"])
    tune_ev  = int(cfg["tune_every"])
    samp_ev  = int(cfg["sample_every"])
    r_maxσ   = float(cfg["r_max"])
    nbins    = int(cfg["gr_bins"])
    seed     = int(cfg.get("seed", 0))

    # --- initialise on lattice ---
    positions, sigma, eta_actual, spacings = initialize_cubic_lattice_2d(N, L, eta)

    # --- RNG and initial step size ---
    rng = np.random.default_rng(seed)
    d0 = 0.2 * sigma  # or choose based on spacing

    # --- equilibration using your existing run_mc (no plotting/frames) ---
    positions_eq, d_after_eq, acc_hist_eq, d_hist_eq, totals_eq = run_mc(
        positions=positions.copy(),
        sigma=sigma,
        L=L,
        d_init=d0,
        equil_steps=eq_steps,
        prod_steps=0,
        tune_every=tune_ev,
        rng=rng,
        plot_every=None,
        save_frames=False,
        frames_dir="frames_mc",
        cap_sigma_frac=0.5,
        stitch_gif=False,
    )
    print(f"[equil] acceptance ~ {100*totals_eq['acceptance_rate']:.1f}%, d = {d_after_eq:.3g}")

    # --- production + g(r) frames ---
    os.makedirs(frames_dir, exist_ok=True)
    positions_pr = positions_eq
    d = float(d_after_eq)

    total_acc = 0
    total_att = 0
    frame_idx = 0

    for step in range(1, pr_steps + 1):
        accepted = mc_step(positions_pr, sigma, L, d, rng)
        total_acc += int(accepted)
        total_att += 1

        # optional: re-use the same tuner as in run_mc
        if step % tune_ev == 0:
            # simple window-free tuner: just use running acceptance
            acc_rate = total_acc / total_att if total_att else 0.0
            d = tune_stepsize(d, acc_rate)   # your original tuner
            # cap step size if desired
            d = min(d, 0.5 * sigma)

        # sample g(r) & save a frame
        if step % samp_ev == 0:
            r_centers, g_r = compute_gr_snapshot(positions_pr, L, sigma, r_maxσ, nbins)

            fig, ax = plt.subplots(figsize=(5, 4))
            ax.plot(r_centers / sigma, g_r)
            ax.set_xlim(0, r_maxσ)
            ax.set_ylim(0, max(3.0, np.max(g_r)*1.1))
            ax.set_xlabel(r"$r/\sigma$")
            ax.set_ylabel(r"$g(r)$")
            ax.set_title(rf"Pair correlation $g(r)$ at $\eta = {eta}$, step = {step}")
            out_path = os.path.join(frames_dir, f"gr_{frame_idx:05d}.png")
            fig.savefig(out_path, dpi=150, bbox_inches="tight")
            plt.close(fig)

            frame_idx += 1

    acc_rate_final = total_acc / total_att if total_att else 0.0
    print(f"[production] acceptance ~ {100*acc_rate_final:.1f}%, frames: {frame_idx}")

    # --- stitch into GIF using your helper ---
    if frame_idx > 0:
        stitch_frames(frames_dir=frames_dir,
                      gif_out=gif_out,
                      mp4_out=None,
                      fps=fps)
        print(f"[done] g(r) animation saved as: {gif_out}")
    else:
        print("[done] no frames saved; GIF not created.")


In [429]:
run_full_sim_and_animate_gr(
    config_path="config.jsonc",
    frames_dir="frames_gr",
    gif_out="gr_evolution.gif",
    fps=10
)


[equil] acceptance ~ 40.0%, d = 1.88
[production] acceptance ~ 43.8%, frames: 500
Saved GIF: gr_evolution.gif
[done] g(r) animation saved as: gr_evolution.gif


In [66]:
def scan_eta_and_compute_gr(base_config: dict,
                            etas: list[float],
                            L: float):
    """
    For each eta in `etas`, run a full MC + g(r) simulation and
    return the curves in a dict keyed by eta.

    Assumes:
      - N = base_config["N"]
      - all the MC / g(r) control parameters (equil_steps, prod_steps, etc.)
        are set in base_config.
      - 2D disks.
    """
    results = {}
    N = base_config["N"]

    for eta in etas:
        print(f"\n=== Running simulation for eta = {eta:.3f} ===")

        # make a shallow copy so we can change eta safely
        cfg = dict(base_config)
        cfg["eta"] = eta

        # particle diameter from eta, N, L
        sigma_eta = sigma_from_eta_2d(N, L, eta)

        # initialise square lattice for this eta
        positions_init, sigma_check, eta_actual, spacings = initialize_cubic_lattice_2d(N, L, eta)
        # (sigma_check should equal sigma_eta up to rounding; we can ignore difference)

        # run equilibration + production + g(r)
        positions_eq, r, g_r = run_full_mc_with_gr(
            config=cfg,
            positions_init=positions_init,
            sigma=sigma_eta,
            L=L,
        )

        results[eta] = {
            "r": r,
            "g_r": g_r,
            "positions_final": positions_eq,
            "sigma": sigma_eta,
            "eta_actual": eta_actual,
        }

    return results


def plot_gr_eta_scan(results: dict,
                     title: str = "Pair correlation function at different $\eta$"):
    """
    Plot g(r) vs r/σ for all eta values stored in `results`.
    `results` is the dict returned by scan_eta_and_compute_gr.
    """
    plt.figure(figsize=(7, 5))

    for eta, data in sorted(results.items()):
        r = data["r"]
        g_r = data["g_r"]
        sigma = data["sigma"]
        r_scaled = r / sigma

        plt.plot(r_scaled, g_r, label=fr"$\eta = {eta:.2f}$")

    plt.xlabel(r"$r/\sigma$")
    plt.ylabel(r"$g(r)$")
    plt.title(title)
    plt.xlim(0.0, 5.0)
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
# base_config is your JSONC-loaded config (for eta you can keep 0.68 or anything;
# we'll overwrite eta per run)
etas = [0.68, 0.69, 0.70, 0.71, 0.72]

gr_results = scan_eta_and_compute_gr(
    base_config=config,
    etas=etas,
    L=L,                 # from your config
)

plot_gr_eta_scan(gr_results,
                 title="Pair correlation functions across the fluid–solid transition")



=== Running simulation for eta = 0.550 ===
Equilibration done.
  Equil acceptance: 40.1%
Production done.
  g(r) sampled 500 times.

=== Running simulation for eta = 0.650 ===
Equilibration done.
  Equil acceptance: 40.0%
Production done.
  g(r) sampled 500 times.

=== Running simulation for eta = 0.750 ===
Equilibration done.
  Equil acceptance: 39.9%
Production done.
  g(r) sampled 500 times.

=== Running simulation for eta = 0.850 ===


ValueError: Infeasible: target η=0.85 ⇒ σ=10.4031, but grid spacing a_min=10 < σ (overlap).
Lower η to ≤ ~0.785398, increase L, or reduce N.